In [1]:
import datetime
import itertools
import json
import time

import pandas as pd
import requests
import streamlit as st


## Gather data using the ShroomDK API

In [3]:
API_KEY = st.secrets["flipside"]["api_key"]
TTL_MINUTES = 60

PAGE_SIZE = 100000
PAGE_NUMBER = 1

In [4]:
statuses = ["TRUE", "FALSE"]
order_vars = ["total_fee", "total_tx"]
join_types = ["left", "inner"]
date_pairs = [
    ("2022-01-01", "2022-02-01"),
    ("2022-02-01", "2022-03-01"),
    ("2022-03-01", "2022-04-01"),
    ("2022-04-01", "2022-05-01"),
    ("2022-05-01", "2022-06-01"),
    ("2022-06-01", "2022-07-01"),
    ("2022-07-01", "2022-08-01"),
    ("2022-08-01", "2022-09-01"),
    ("2022-09-01", f"{datetime.date.today():%Y-%m-%d}"),
]
dates = [
    f"{x:%Y-%m-%d}"
    for x in pd.date_range(datetime.date(2022, 1, 1), datetime.datetime.today())
]


In [5]:
def create_query(query, ttl_min, api_key):
    r = requests.post(
        "https://node-api.flipsidecrypto.com/queries",
        data=json.dumps({"sql": query, "ttlMinutes": ttl_min}),
        headers={
            "Accept": "application/json",
            "Content-Type": "application/json",
            "x-api-key": api_key,
        },
    )
    if r.status_code != 200:
        raise Exception(
            "Error creating query, got response: "
            + r.text
            + "with status code: "
            + str(r.status_code)
        )

    return json.loads(r.text)

def submit_queries(
    template,
    sleep_time=2,
    **kwargs,
):
    query_dict = {}
    redos = {}

    template_name = template.split("@")[1][6:]
    print(f"Working on {template_name}...")
    combos = [dict(zip(kwargs.keys(), x)) for x in itertools.product(*kwargs.values())]

    for i, x in enumerate(combos.copy()):
        for k, v in x.copy().items():
            if type(v) == tuple:
                pairs = combos[i].pop(k)
                for j, y in enumerate(pairs):
                    combos[i][f"{k}_{j}"] = y
    print(f"Submitting {len(combos)} queries...")
    queries = {}
    for i, x in enumerate(combos):
        # for k,v in x:
        if i % 10 == 0:
            print(f"Submitting job {i}/{len(combos)}")
        name = f"{template_name}__{'--'.join(x.values())}"
        qry = template.format(**x)
        queries[name] = {"query": qry}
        try:
            q = create_query(qry, TTL_MINUTES, API_KEY)
            token = q.get("token")
            time.sleep(sleep_time)
            query_dict[name] = {"query": qry, "token": token}
        except Exception:  # reattempt run
            try:
                time.sleep(5)
                q = create_query(qry, TTL_MINUTES, API_KEY)
                token = q.get("token")
                time.sleep(sleep_time)
                query_dict[name] = {"query": qry, "token": token}
            except Exception as e:
                print(f"This query is not submitted: {name}")
                redos[name] = {"exception": e, "query": qry}
                time.sleep(sleep_time)
    return query_dict, redos



def check_query(token, api_key):
    r = requests.get(
       f'https://node-api.flipsidecrypto.com/queries/{token}?pageNumber={PAGE_NUMBER}&pageSize={PAGE_SIZE}',
        headers={
            "Accept": "application/json",
            "Content-Type": "application/json",
            "x-api-key": api_key,
        },
    )
    if r.status_code != 200:
        raise Exception(
            "Error getting query results, got response: "
            + r.text
            + "with status code: "
            + str(r.status_code)
        )

    data = json.loads(r.text)
    return data

In [6]:
all_tps_info_template = """
--sql @name: all_tps_info@
-- TODO: use Kida's regex?
with consumption_tx as (
    select
        t.block_timestamp,
        t.tx_id,
        t.fee,
        t.succeeded,
        sum(
            split(
                regexp_substr(s.value, '[0-9]* of [0-9]*'),
                ' of '
            ) [0] :: int
        ) as compute_units_used,
        avg(
            split(
                regexp_substr(s.value, '[0-9]* of [0-9]*'),
                ' of '
            ) [1] :: int
        ) as avg_compute_units_requested,
        avg(
            split(
                regexp_substr(s.value, '[0-9]* of [0-9]*'),
                ' of '
            ) [0] :: int / split(
                regexp_substr(s.value, '[0-9]* of [0-9]*'),
                ' of '
            ) [1] :: int
        ) as avg_compute_units_proportion
    from
        solana.core.fact_transactions t,
        lateral flatten(input => t.log_messages) s
    where
        block_timestamp :: date = '{date}'
        and s.value like '% consumed %'
    group by
        t.block_timestamp,
        t.tx_id,
        t.fee,
        t.succeeded
)
select
    date_trunc('hour', block_timestamp) as datetime,
    -- total tx
    count(tx_id) as total_tx,
    sum(fee) as total_fee,
    avg(fee) as avg_total_fee,
    sum(compute_units_used) as total_compute_units_used,
    avg(compute_units_used) as total_avg_compute_units_used,
    avg(avg_compute_units_requested) as total_avg_compute_units_requested,
    avg(avg_compute_units_proportion) as total_avg_compute_units_proportion,
    -- successful tx:
    count(
        case
            when succeeded = 'TRUE' then succeeded
            else NULL
        end
    ) as successful_tx,
    sum(
        case
            when succeeded = 'TRUE' then fee
            else NULL
        end
    ) as successful_fee,
    avg(
        case
            when succeeded = 'TRUE' then fee
            else NULL
        end
    ) as avg_successful_fee,
    sum(
        case
            when succeeded = 'TRUE' then compute_units_used
            else NULL
        end
    ) as successful_compute_units_used,
    avg(
        case
            when succeeded = 'TRUE' then compute_units_used
            else NULL
        end
    ) as avg_successful_compute_units_used,
    avg(
        case
            when succeeded = 'TRUE' then avg_compute_units_requested
            else NULL
        end
    ) as avg_successful_compute_units_requested,
    avg(
        case
            when succeeded = 'TRUE' then avg_compute_units_proportion
            else NULL
        end
    ) as avg_successful_compute_units_proportion,
    -- failed tx:
    count(
        case
            when succeeded = 'FALSE' then succeeded
            else NULL
        end
    ) as failed_tx,
    sum(
        case
            when succeeded = 'FALSE' then fee
            else NULL
        end
    ) as failed_fee,
    avg(
        case
            when succeeded = 'FALSE' then fee
            else NULL
        end
    ) as avg_failed_fee,
    sum(
        case
            when succeeded = 'FALSE' then compute_units_used
            else NULL
        end
    ) as failed_compute_units_used,
    avg(
        case
            when succeeded = 'FALSE' then compute_units_used
            else NULL
        end
    ) as avg_failed_compute_units_used,
    avg(
        case
            when succeeded = 'FALSE' then avg_compute_units_requested
            else NULL
        end
    ) as avg_failed_compute_units_requested,
    avg(
        case
            when succeeded = 'FALSE' then avg_compute_units_proportion
            else NULL
        end
    ) as avg_failed_compute_units_proportion,
    -- rates:
    successful_tx / total_tx as success_rate,
    total_tx / 3600 as total_tps,
    successful_tx / 3600 as succesful_tps,
    failed_tx / 3600 as failed_tps
from
    consumption_tx
group by
    datetime
order by
    datetime 
--end-sql
"""

In [7]:
queries, redos = submit_queries(
    all_tps_info_template,
    0,
    date=dates
)

Working on all_tps_info...
Submitting 272 queries...
Submitting job 0/272
Submitting job 10/272
Submitting job 20/272
Submitting job 30/272
Submitting job 40/272
Submitting job 50/272
Submitting job 60/272
Submitting job 70/272
Submitting job 80/272
Submitting job 90/272
Submitting job 100/272
Submitting job 110/272
Submitting job 120/272
Submitting job 130/272
Submitting job 140/272
Submitting job 150/272
Submitting job 160/272
Submitting job 170/272
Submitting job 180/272
Submitting job 190/272
Submitting job 200/272
Submitting job 210/272
Submitting job 220/272
Submitting job 230/272
Submitting job 240/272
Submitting job 250/272
Submitting job 260/272
Submitting job 270/272


In [8]:
len(queries), len(dates)

(272, 272)

In [9]:
with open('data/queries.json', 'w') as f:
    json.dump(queries, f)

In [10]:
time.sleep(600) # give everything 10+ minutes to run
data_dict = {}
running = {}
for k, v in queries.items():
    try:
        result = check_query(v["token"], API_KEY)
        if result["status"] == "running":
            running[k] = v
        else:
            data_dict[k] = result
    except Exception as e:
        print(f"{k}: {e}")

all_tps_info__2022-01-08: Error getting query results, got response: {"errors":"Division by zero"}with status code: 400
all_tps_info__2022-02-08: Error getting query results, got response: {"errors":"Division by zero"}with status code: 400
all_tps_info__2022-02-10: Error getting query results, got response: {"errors":"Division by zero"}with status code: 400
all_tps_info__2022-02-20: Error getting query results, got response: {"errors":"Division by zero"}with status code: 400
all_tps_info__2022-03-02: Error getting query results, got response: {"errors":"Division by zero"}with status code: 400
all_tps_info__2022-03-07: Error getting query results, got response: {"errors":"Division by zero"}with status code: 400
all_tps_info__2022-03-29: Error getting query results, got response: {"errors":"Division by zero"}with status code: 400
all_tps_info__2022-05-04: Error getting query results, got response: {"errors":"Division by zero"}with status code: 400
all_tps_info__2022-05-09: Error getting 

Not sure exactly what is causing this division by 0 error.
This is a SQL error which occurs on Velocity as well for these dates.
Either `total_tx` (in `successful_tx / total_tx as success_rate`), or
```sql
split(
    regexp_substr(s.value, '[0-9]* of [0-9]*'),
    ' of '
) [1] :: int
```
is 0 for one of the hours during that day

In [11]:
len(running), len(data_dict)

(0, 234)

In [12]:
with open('data/data.json', 'w') as f:
    json.dump(data_dict, f)

In [14]:
(len(data_dict) - len(dates)) / len(dates)

-0.13970588235294118

### Save data to dataframe

In [15]:
dfs_tps = []
for k in data_dict:
    if "all" in k:
        query_data = data_dict[k]
        df = pd.DataFrame(query_data["results"], columns=query_data["columnLabels"])
        dfs_tps.append(df)


In [16]:
df = pd.concat(dfs_tps).sort_values(by='DATETIME').reset_index(drop=True)
df['DATETIME'] = pd.to_datetime(df.DATETIME)
df = df[df['DATETIME'] < pd.to_datetime(datetime.date.today())]

In [17]:
for x in df.columns:
    if 'FEE' in x:
        df[x] = df[x] / 1_000_000_000 # convert to SOL

In [18]:
df.to_csv('data/tps.csv', index=False)

## Investigate how to use data

In [19]:
df = pd.read_csv("data/tps.csv")

In [20]:
df.columns

Index(['DATETIME', 'TOTAL_TX', 'TOTAL_FEE', 'AVG_TOTAL_FEE',
       'TOTAL_COMPUTE_UNITS_USED', 'TOTAL_AVG_COMPUTE_UNITS_USED',
       'TOTAL_AVG_COMPUTE_UNITS_REQUESTED',
       'TOTAL_AVG_COMPUTE_UNITS_PROPORTION', 'SUCCESSFUL_TX', 'SUCCESSFUL_FEE',
       'AVG_SUCCESSFUL_FEE', 'SUCCESSFUL_COMPUTE_UNITS_USED',
       'AVG_SUCCESSFUL_COMPUTE_UNITS_USED',
       'AVG_SUCCESSFUL_COMPUTE_UNITS_REQUESTED',
       'AVG_SUCCESSFUL_COMPUTE_UNITS_PROPORTION', 'FAILED_TX', 'FAILED_FEE',
       'AVG_FAILED_FEE', 'FAILED_COMPUTE_UNITS_USED',
       'AVG_FAILED_COMPUTE_UNITS_USED', 'AVG_FAILED_COMPUTE_UNITS_REQUESTED',
       'AVG_FAILED_COMPUTE_UNITS_PROPORTION', 'SUCCESS_RATE', 'TOTAL_TPS',
       'SUCCESFUL_TPS', 'FAILED_TPS'],
      dtype='object')

In [21]:
tps_df = df[
    [
        "DATETIME",
        "TOTAL_TPS",
        "SUCCESFUL_TPS",
        "FAILED_TPS",
    ]
].melt(id_vars="DATETIME")
fee_df = df[
    [
        "DATETIME",
        "TOTAL_FEE",
        "SUCCESSFUL_FEE",
        "FAILED_FEE",
    ]
].melt(id_vars="DATETIME")
avg_fee_df = df[
    [
        "DATETIME",
        "AVG_TOTAL_FEE",
        "AVG_SUCCESSFUL_FEE",
        "AVG_FAILED_FEE",
    ]
].melt(id_vars="DATETIME")
tx_df = df[
    [
        "DATETIME",
        "TOTAL_TX",
        "SUCCESSFUL_TX",
        "FAILED_TX",
    ]
].melt(id_vars="DATETIME")
compute_units_used_df = df[
    [
        "DATETIME",
        "TOTAL_COMPUTE_UNITS_USED",
        "SUCCESSFUL_COMPUTE_UNITS_USED",
        "FAILED_COMPUTE_UNITS_USED",
    ]
].melt(id_vars="DATETIME")
avg_compute_units_used_df = df[
    [
        "DATETIME",
        "TOTAL_AVG_COMPUTE_UNITS_USED",
        "AVG_SUCCESSFUL_COMPUTE_UNITS_USED",
        "AVG_FAILED_COMPUTE_UNITS_USED",
    ]
].melt(id_vars="DATETIME")
avg_compute_units_requested_df = df[
    [
        "DATETIME",
        "TOTAL_AVG_COMPUTE_UNITS_REQUESTED",
        "AVG_SUCCESSFUL_COMPUTE_UNITS_REQUESTED",
        "AVG_FAILED_COMPUTE_UNITS_REQUESTED",
    ]
].melt(id_vars="DATETIME")
avg_compute_units_proportion_df = df[
    [
        "DATETIME",
        "TOTAL_AVG_COMPUTE_UNITS_PROPORTION",
        "AVG_SUCCESSFUL_COMPUTE_UNITS_PROPORTION",
        "AVG_FAILED_COMPUTE_UNITS_PROPORTION",
    ]
].melt(id_vars="DATETIME")


In [22]:
tps_df = df[
    [
        "DATETIME",
        "TOTAL_TPS",
        "SUCCESFUL_TPS",
        "FAILED_TPS",
    ]
]

In [23]:
tps_df

,DATETIME,TOTAL_TPS,SUCCESFUL_TPS,FAILED_TPS
0,2022-01-01 00:00:00,816.017778,624.088056,191.929722
1,2022-01-01 01:00:00,806.275833,614.596667,191.679167
2,2022-01-01 02:00:00,675.293056,463.925278,211.367778
3,2022-01-01 03:00:00,777.086111,567.031944,210.054167
4,2022-01-01 04:00:00,833.851111,626.094167,207.756944
...,...,...,...,...
5583,2022-09-28 19:00:00,445.218889,352.172222,93.046667
5584,2022-09-28 20:00:00,439.984444,351.374167,88.610278
5585,2022-09-28 21:00:00,389.107500,311.995556,77.111944
5586,2022-09-28 22:00:00,390.909444,311.299722,79.609722


In [24]:
tps_df.rolling(24).mean()

/tmp/ipykernel_425556/3233464772.py:1: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['DATETIME'], dtype='object')
  tps_df.rolling(24).mean()


,TOTAL_TPS,SUCCESFUL_TPS,FAILED_TPS
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
...,...,...,...
5583,425.890162,335.852083,90.038079
5584,424.571539,335.011447,89.560093
5585,422.779306,333.119699,89.659606
5586,421.405613,332.071007,89.334606


In [25]:
avg_fee_df.melt()

/tmp/ipykernel_425556/4250769966.py:1: FutureWarning: This dataframe has a column name that matches the 'value_name' column name of the resulting Dataframe. In the future this will raise an error, please set the 'value_name' parameter of DataFrame.melt to a unique name.
  avg_fee_df.melt()


,variable,value
0,DATETIME,2022-01-01 00:00:00
1,DATETIME,2022-01-01 01:00:00
2,DATETIME,2022-01-01 02:00:00
3,DATETIME,2022-01-01 03:00:00
4,DATETIME,2022-01-01 04:00:00
...,...,...
50287,value,0.00001
50288,value,0.000008
50289,value,0.000008
50290,value,0.000008
